In [ ]:
import datetime
from typing import List, Dict, Union

class MPINValidator:
    COMMON_WEAK_PINS_4_DIGIT = [
        '1234', '4321', '0000', '1111', '1122', 
        '2222', '3333', '4444', '5555', '6666', 
        '7777', '8888', '9999', '1212', '2121'
    ]
    
    COMMON_WEAK_PINS_6_DIGIT = [
        '123456', '654321', '000000', '111111', '222222', 
        '333333', '444444', '555555', '666666', '777777', 
        '888888', '999999', '121212', '212121'
    ]
    
    def validate_mpin(
        mpin: str, 
        user_demographics: Dict[str, Union[str, datetime.date]] = None
    ) -> Dict[str, Union[str, List[str]]]:
        # Input validation
        if not mpin or not mpin.isdigit():
            raise ValueError("MPIN must be a non-empty string of digits")
        
        # Validate PIN length
        if len(mpin) not in [4, 6]:
            raise ValueError("MPIN must be either 4 or 6 digits long")
        
        # Select common weak PINs based on length
        common_weak_pins = (
            MPINValidator.COMMON_WEAK_PINS_6_DIGIT if len(mpin) == 6 
            else MPINValidator.COMMON_WEAK_PINS_4_DIGIT
        )
        
        #Initialize result
        result = {
            'strength': 'STRONG',
            'reasons': []
        }
        
        # Check if MPIN is in common weak PINs
        if mpin in common_weak_pins:
            result['strength'] = 'WEAK'
            result['reasons'].append('COMMONLY_USED')
        
        # Check demographics if provided
        if user_demographics:
            demographic_reasons = MPINValidator._check_demographics(mpin, user_demographics)
            if demographic_reasons:
                result['strength'] = 'WEAK'
                result['reasons'].extend(demographic_reasons)
        
        return result
    
    
    def _check_demographics(
        mpin: str, 
        demographics: Dict[str, Union[str, datetime.date]]
    ) -> List[str]:
        reasons = []
        
        # Extract dates from demographics
        dob_self = demographics.get('dob_self')
        dob_spouse = demographics.get('dob_spouse')
        anniversary = demographics.get('anniversary')
        
        #Convert dates to MPIN type
        def date_to_pin_formats(date: datetime.date) -> List[str]:
            if not date:
                return []
            return [
                f"{date.day:02d}{date.month:02d}",
                f"{date.month:02d}{date.day:02d}",
                f"{date.year % 100:02d}{date.month:02d}",
                f"{date.month:02d}{date.year % 100:02d}",
                f"{date.year % 100:02d}{date.day:02d}",
                f"{date.day:02d}{date.year % 100:02d}"
            ]
        
        # Check against self DOB
        if dob_self and isinstance(dob_self, datetime.date):
            dob_formats = date_to_pin_formats(dob_self)
            if any(pin_format in mpin for pin_format in dob_formats):
                reasons.append('DEMOGRAPHIC_DOB_SELF')
        
        # Check against spouse DOB
        if dob_spouse and isinstance(dob_spouse, datetime.date):
            spouse_dob_formats = date_to_pin_formats(dob_spouse)
            if any(pin_format in mpin for pin_format in spouse_dob_formats):
                reasons.append('DEMOGRAPHIC_DOB_SPOUSE')
        
        # Check against anniversary
        if anniversary and isinstance(anniversary, datetime.date):
            anniversary_formats = date_to_pin_formats(anniversary)
            if any(pin_format in mpin for pin_format in anniversary_formats):
                reasons.append('DEMOGRAPHIC_ANNIVERSARY')
        
        return reasons

def run_comprehensive_tests():
    test_cases = [
        # Part A: Common weak PINs
        {'mpin': '1234', 'expected': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
        {'mpin': '9876', 'expected': {'strength': 'STRONG', 'reasons': []}},
        
        # Part B & C: Demographic checks
        {
            'mpin': '0201', 
            'demographics': {
                'dob_self': datetime.date(1998, 2, 1)
            },
            'expected': {
                'strength': 'WEAK', 
                'reasons': ['DEMOGRAPHIC_DOB_SELF']
            }
        },
        {
            'mpin': '0602', 
            'demographics': {
                'dob_spouse': datetime.date(1990, 6, 2)
            },
            'expected': {
                'strength': 'WEAK', 
                'reasons': ['DEMOGRAPHIC_DOB_SPOUSE']
            }
        },
        {
            'mpin': '1205', 
            'demographics': {
                'anniversary': datetime.date(2015, 5, 12)
            },
            'expected': {
                'strength': 'WEAK', 
                'reasons': ['DEMOGRAPHIC_ANNIVERSARY']
            }
        },
        
        # Part D: 6-digit PIN checks
        {'mpin': '123456', 'expected': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
        {'mpin': '987654', 'expected': {'strength': 'STRONG', 'reasons': []}},
        
        {
            'mpin': '020190', 
            'demographics': {
                'dob_self': datetime.date(1998, 2, 1),
                'dob_spouse': datetime.date(1990, 6, 2)
            },
            'expected': {
                'strength': 'WEAK', 
                'reasons': ['DEMOGRAPHIC_DOB_SELF', 'DEMOGRAPHIC_DOB_SPOUSE']
            }
        },
        
        # Multiple demographic matches
        {
            'mpin': '120515', 
            'demographics': {
                'dob_self': datetime.date(2015, 5, 12),
                'anniversary': datetime.date(2015, 5, 12)
            },
            'expected': {
                'strength': 'WEAK', 
                'reasons': ['DEMOGRAPHIC_DOB_SELF', 'DEMOGRAPHIC_ANNIVERSARY']
            }
        },
        
        # Edge cases and error scenarios
        {'mpin': '1111', 'expected': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
        {'mpin': '9999', 'expected': {'strength': 'WEAK', 'reasons': ['COMMONLY_USED']}},
    ]
    
    # Run tests
    total_tests = len(test_cases)
    passed_tests = 0
    
    print("Running Comprehensive MPIN Validator Tests:")
    for i, case in enumerate(test_cases, 1):
        try:
            mpin = case['mpin']
            demographics = case.get('demographics', {})
            
            result = MPINValidator.validate_mpin(mpin, demographics)
            expected = case['expected']
            
            if (result['strength'] == expected['strength'] and 
                set(result['reasons']) == set(expected['reasons'])):
                print(f"Test Case {i}: PASSED")
                passed_tests += 1
            else:
                print(f"Test Case {i}: FAILED")
                print(f"  MPIN: {mpin}")
                print(f"  Demographics: {demographics}")
                print(f"  Expected: {expected}")
                print(f"  Got:      {result}")
        
        except Exception as e:
            print(f"Test Case {i}: ERROR - {str(e)}")
    
    print(f"\nTest Summary: {passed_tests}/{total_tests} tests passed")

def interactive_mpin_checker():
    print("MPIN Strength Validator")
    print("---------------------")
    print("Commands:")
    print("  - Enter MPIN to check strength")
    print("  - Type 'exit' or 'quit' to stop the entire operation")
    
    while True:
        try:
            user_input = input("\nEnter MPIN (4 or 6 digits) or command: ").strip().lower()
            
            if user_input in ['exit', 'quit', 'q']:
                print("Exiting MPIN Strength Validator. Goodbye!")
                break
            
            result = MPINValidator.validate_mpin(user_input)
            
            print("Optional: Enter demographic information (leave blank if not applicable)")
            dob_self_str = input("Your Date of Birth (YYYY-MM-DD): ").strip()
            
            if dob_self_str.lower() in ['exit', 'quit', 'q']:
                print("Exiting MPIN Strength Validator. Goodbye!")
                break
            
            dob_spouse_str = input("Spouse's Date of Birth (YYYY-MM-DD): ").strip()
            
            if dob_spouse_str.lower() in ['exit', 'quit', 'q']:
                print("Exiting MPIN Strength Validator. Goodbye!")
                break
            
            anniversary_str = input("Wedding Anniversary (YYYY-MM-DD): ").strip()
            
            if anniversary_str.lower() in ['exit', 'quit', 'q']:
                print("Exiting MPIN Strength Validator. Goodbye!")
                break
            
            demographics = {}
            if dob_self_str:
                demographics['dob_self'] = datetime.datetime.strptime(dob_self_str, "%Y-%m-%d").date()
            if dob_spouse_str:
                demographics['dob_spouse'] = datetime.datetime.strptime(dob_spouse_str, "%Y-%m-%d").date()
            if anniversary_str:
                demographics['anniversary'] = datetime.datetime.strptime(anniversary_str, "%Y-%m-%d").date()
            

            result = MPINValidator.validate_mpin(user_input, demographics)
            
            
            print("\nMPIN Strength Analysis:")
            print(f"Strength: {result['strength']}")
            if result['reasons']:
                print("Weakness Reasons:")
                for reason in result['reasons']:
                    print(f"  - {reason}")
        
        except ValueError as e:
            print(f"Error: {e}")
            print("Please try again or type 'exit' to quit.")
        except Exception as e:
            print(f"Unexpected error: {e}")
            print("Please try again or type 'exit' to quit.")

if __name__ == '__main__':
    interactive_mpin_checker()

MPIN Strength Validator
---------------------
Commands:
  - Enter MPIN to check strength
  - Type 'exit' or 'quit' to stop the entire operation
Optional: Enter demographic information (leave blank if not applicable)
Exiting MPIN Strength Validator. Goodbye!
